## Въведение

Този урок ще обхване:
- Какво е извикване на функция и кога се използва
- Как да създадете извикване на функция с OpenAI
- Как да интегрирате извикване на функция в приложение

## Учебни цели

След като завършите този урок, ще знаете и разбирате:

- Защо се използва извикване на функция
- Как да настроите извикване на функция чрез OpenAI Service
- Как да проектирате ефективни извиквания на функции според нуждите на вашето приложение


## Разбиране на извикванията на функции

В този урок ще изградим функционалност за нашия образователен стартъп, която позволява на потребителите да използват чатбот, за да намират технически курсове. Ще препоръчваме курсове, които отговарят на тяхното ниво на умения, текуща роля и интересуваща ги технология.

За да реализираме това, ще използваме комбинация от:
 - `OpenAI` за създаване на чат изживяване за потребителя
 - `Microsoft Learn Catalog API`, за да помагаме на потребителите да намират курсове според тяхното запитване
 - `Function Calling`, за да вземем заявката на потребителя и да я изпратим към функция, която да направи API заявка.

За да започнем, нека видим защо изобщо бихме искали да използваме извикване на функции:

print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # получаваме нов отговор от GPT, при който той може да види отговора на функцията


print(second_response.choices[0].message)


### Защо да използваме Function Calling

Ако сте преминали през някой друг урок от този курс, вероятно вече разбирате колко мощни са големите езикови модели (LLMs). Надявам се също така да сте забелязали и някои от техните ограничения.

Function Calling е функция на OpenAI Service, създадена да реши следните предизвикателства:

Непоследователен формат на отговорите:
- Преди function calling, отговорите от големите езикови модели бяха неструктурирани и непоследователни. Разработчиците трябваше да пишат сложен код за валидиране, за да обработят всяка вариация в изхода.

Ограничена интеграция с външни данни:
- Преди тази функция беше трудно да се включат данни от други части на приложението в чат контекста.

Като стандартизира формата на отговорите и позволява лесна интеграция с външни данни, function calling улеснява разработката и намалява нуждата от допълнителна логика за валидиране.

Потребителите не можеха да получат отговори като "Какво е времето в момента в Стокхолм?". Причината е, че моделите са ограничени до момента, в който са били обучени с данни.

Нека разгледаме примера по-долу, който илюстрира този проблем:

Да кажем, че искаме да създадем база данни с данни за студенти, за да можем да им препоръчаме подходящ курс. По-долу имаме две описания на студенти, които са много сходни по информацията, която съдържат.


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finshing his studies."

Искаме да изпратим това към LLM, за да анализира данните. По-късно това може да се използва в нашето приложение, за да се изпрати към API или да се съхрани в база данни.

Нека създадем два еднакви подканващи текста, с които инструктираме LLM каква информация ни интересува:


Искаме да изпратим това на LLM, за да анализира частите, които са важни за нашия продукт. Така че можем да създадем два еднакви подканващи текста, за да инструктираме LLM:


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


След като създадем тези два подканващи текста, ще ги изпратим към LLM, използвайки `openai.ChatCompletion`. Съхраняваме подканващия текст в променливата `messages` и задаваме ролята на `user`. Това е с цел да се имитира съобщение от потребител, написано към чатбот.


In [ ]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

deployment="gpt-3.5-turbo"

: 

Сега можем да изпратим и двата заявки към LLM и да разгледаме получения отговор.


In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

Въпреки че подканите са еднакви и описанията са сходни, можем да получим различни формати на свойството `Grades`.

Ако изпълните горната клетка няколко пъти, форматът може да бъде `3.7` или `3.7 GPA`.

Това е така, защото LLM приема неструктурирани данни под формата на написаната подканя и също така връща неструктурирани данни. Необходимо е да имаме структуриран формат, за да знаем какво да очакваме, когато съхраняваме или използваме тези данни.

Чрез използване на функционално извикване можем да сме сигурни, че ще получим обратно структурирани данни. Когато използваме функционално извикване, LLM всъщност не извиква и не изпълнява никакви функции. Вместо това създаваме структура, която LLM да следва в отговорите си. След това използваме тези структурирани отговори, за да знаем коя функция да изпълним в нашите приложения.


![Диаграма на потока на извикване на функция](../../../../translated_images/Function-Flow.083875364af4f4bb69bd6f6ed94096a836453183a71cf22388f50310ad6404de.bg.png)


### Примери за използване на извикване на функции

**Извикване на външни инструменти**  
Чатботовете са отлични в предоставянето на отговори на въпроси от потребителите. Чрез използване на извикване на функции, чатботовете могат да използват съобщенията от потребителите, за да изпълнят определени задачи. Например, студент може да помоли чатбота: "Изпрати имейл на моя преподавател, че имам нужда от повече помощ по този предмет". Това може да извика функцията `send_email(to: string, body: string)`

**Създаване на API или заявки към база данни**  
Потребителите могат да намират информация, използвайки естествен език, който се преобразува във форматирана заявка или API заявка. Пример за това е учител, който пита: "Кои са учениците, които завършиха последното задание", което може да извика функция с име `get_completed(student_name: string, assignment: int, current_status: string)`

**Създаване на структурирани данни**  
Потребителите могат да вземат текстов блок или CSV и да използват LLM, за да извлекат важна информация от него. Например, студент може да преобразува статия от Уикипедия за мирни споразумения, за да създаде AI флашкарти. Това може да се направи чрез функцията `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)`


## 2. Създаване на първото ви извикване на функция

Процесът на създаване на извикване на функция включва 3 основни стъпки:
1. Извикване на Chat Completions API с списък от вашите функции и съобщение от потребителя
2. Прочитане на отговора на модела, за да се извърши действие, например изпълнение на функция или API заявка
3. Още едно извикване към Chat Completions API с отговора от вашата функция, за да използвате тази информация за създаване на отговор към потребителя.


![Поток на извикване на функция](../../../../translated_images/LLM-Flow.3285ed8caf4796d7343c02927f52c9d32df59e790f6e440568e2e951f6ffa5fd.bg.png)


### Елементи на извикване на функция

#### Вход от потребителя

Първата стъпка е да създадете съобщение от потребителя. Това може да се зададе динамично чрез стойността на текстово поле или можете да зададете стойност тук. Ако за първи път работите с Chat Completions API, трябва да определим `role` и `content` на съобщението.

`role` може да бъде `system` (за създаване на правила), `assistant` (моделът) или `user` (крайният потребител). За извикване на функция ще го зададем като `user` и ще дадем примерен въпрос.


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### Създаване на функции.

Следва да дефинираме функция и нейните параметри. Тук ще използваме само една функция, наречена `search_courses`, но можете да създадете и повече функции.

**Важно**: Функциите се включват в системното съобщение към LLM и ще бъдат част от броя налични токени, с които разполагате.


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**Дефиниции**

Структурата на дефиницията на функция има няколко нива, като всяко от тях има свои свойства. Ето как изглежда вложената структура:

**Свойства на функцията на най-високо ниво:**

`name` - Името на функцията, която искаме да бъде извикана.

`description` - Описание на начина, по който работи функцията. Тук е важно да бъдете конкретни и ясни.

`parameters` - Списък с стойности и формат, които искате моделът да използва в отговора си.

**Свойства на обекта Parameters:**

`type` - Типът данни на обекта parameters (обикновено "object")

`properties` - Списък със специфичните стойности, които моделът ще използва за отговора си.

**Свойства на отделен параметър:**

`name` - Неявно дефинирано чрез ключа на свойството (например "role", "product", "level")

`type` - Типът данни на конкретния параметър (например "string", "number", "boolean")

`description` - Описание на конкретния параметър

**Опционални свойства:**

`required` - Масив, който изброява кои параметри са задължителни, за да бъде извикването на функцията успешно


### Извикване на функцията
След като сме дефинирали функция, сега трябва да я включим в извикването към Chat Completion API. Това става като добавим `functions` към заявката. В този случай използваме `functions=functions`.

Има и опция да зададете `function_call` на `auto`. Това означава, че ще оставим LLM да реши коя функция да извика според съобщението на потребителя, вместо ние да я избираме.


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

Сега нека разгледаме отговора и да видим как е форматиран:

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

Може да видите, че името на функцията е извикано и от съобщението на потребителя LLM е успял да намери данните, които да попълни като аргументи на функцията.


## 3. Интегриране на извиквания на функции в приложение.

След като тествахме форматирания отговор от LLM, сега можем да го интегрираме в приложение.

### Управление на процеса

За да го интегрираме в нашето приложение, нека следваме следните стъпки:

Първо, нека направим заявка към OpenAI услугите и запазим съобщението в променлива, наречена `response_message`.


In [ ]:
response_message = response.choices[0].message

Сега ще дефинираме функцията, която ще извика Microsoft Learn API, за да получи списък с курсове:


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



Като добра практика, ще проверим дали моделът иска да извика функция. След това ще създадем една от наличните функции и ще я свържем с функцията, която се извиква.
След това ще вземем аргументите на функцията и ще ги съпоставим с аргументите от LLM.

Накрая ще добавим съобщението за извикване на функцията и стойностите, които са върнати от съобщението `search_courses`. Това дава на LLM цялата необходима информация,
за да отговори на потребителя на естествен език.


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## Предизвикателство с код

Страхотна работа! За да продължите с ученето за OpenAI Function Calling, можете да изградите: https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst
 - Още параметри на функцията, които могат да помогнат на обучаващите се да намерят повече курсове. Можете да намерите наличните API параметри тук:
 - Създайте още едно извикване на функция, което взема повече информация от обучаващия се, например неговия роден език
 - Добавете обработка на грешки, когато извикването на функцията и/или API не връща подходящи курсове



---

**Отказ от отговорност**:  
Този документ е преведен с помощта на AI услуга за превод [Co-op Translator](https://github.com/Azure/co-op-translator). Въпреки че се стремим към точност, имайте предвид, че автоматизираните преводи може да съдържат грешки или неточности. Оригиналният документ на неговия изходен език трябва да се счита за авторитетен източник. За критична информация се препоръчва професионален човешки превод. Не носим отговорност за недоразумения или погрешни тълкувания, произтичащи от използването на този превод.
